[Tensorflow](https://www.tensorflow.org) is an open source library for numerical computation.
Tensorflow implements a symbolic math library that is used to build a computation graph.
This graph is a representation of the computations we wish to perform.
When we write a graph with TensorFlow it is like writing an equation in paper, there are no actual computations occurring.
TensorFlow uses the graph to automatically compute the derivatives required for stochastic gradient descent.
When it is time to actually perform the computations, TensorFlow uses algorithms in C and C++ for it.




# Install



Tensorflow can be installed with `pip`.
After activating your environment in the terminal, run:



Now `numpy` is available for Python and can be imported with:



All of the Tensorflow functionality can now be accessed via the object `tensorflow`.

You will often see the following command being used:



Which makes the `tensorflow` functionality available via `tf`.
This is often used because it requires less typing to use numpy.




# Building the Graph



First import TensorFlow:



A default graph which is empty is automatically created when we first import TensorFlow. We can access this default graph via:



We can see what operations are attached to it (none in this case) with:



Remember that when we add operations to this graph (think about the nodes and connections in a neural network graph) nothing is actually being computed, we are just creating the structure of the function.

The simplest operation we can add is a constant:



Notice that by calling `tf.constant`, TensorFlow automatically adds a constant to the default graph, which we can see was added with the `get_operations` method.
We did not need to explicitly tell TensorFlow which graph to alter.
We can specify which graph to alter, but usually the default graph will suffice.

Each operation in TensorFlow takes an input and generates an output.
Think about the signal $s_k$ in the neural network graph. It takes as input the features $x_1,\dots,x_n$, and outputs the value of the signal $s_k$.
A constant has no inputs, but only an output.
We can see those values via:



The output has a special type:



The type is `tf.Tensor`, which can represent data in ascalar, vector, matrix or a multidimensional array.
The `shape` is `()` which implies this tensor represents a constant.

All tensors are outputs of some operation in the graph.
In this case, the tensor above is the output of a "constant operation".

Let's add in another constant:



Now there are two operations in the graph.
Notice each has a name, "Const" and "Const<sub>1</sub>".
Also observe the types are `tf.Operation`.

Let's create a new operation that will sum up the two constants.
First, we need to give names to the outputs of the two constants, so that the addition knows what to add:



The naming of the tensors follows `operation_name:output_number`.
So "Const:0" means the 0th output (first output) coming out of the operation "Const".

We can combine these outputs:



Important, when we create a new operation and assign it to a variable, like `sum_tensor` above, the variable gets the output of the operation, not the operation itself. That is, `sum_tensor` is already the output of `tf.add`.

We can verify that the addition operation takes two inputs and generates one output:




# Running the Computations



Let's now run this computation.
To do so, we need to create a `Session` object.



The Session object stores a reference to the graph we just constructed, determines how much memory it will need and sets up some other configurations.

We can see that the default graph is indeed being used via:



There is a method `Session.run` which sends the graph to the computation engine to be executed.
The method takes as input the tensor you want to compute.
If we want to compute the tensor of the addition, we would run:



You can also pass a list of tensors to compute.
This is more efficient, because each time we call the `Session.run` method all computations are performed again.



After we are done with the computations, we can close the session to free resources:




# Placeholders



The graph we built uses the same two constant values as inputs to the addition operation.
Let's modify it so that it can take variables as inputs.
In tensorflow variables are created with `tf.placeholder`:



The command above adds a placeholder to the default graph.
It takes as input the type of the variable, in this case it is a 32 bytes float.
Other possible types are: `tf.int32` and `tf.bool`.
A complete list is available [here](https://www.tensorflow.org/api_docs/python/tf/DType).

Let's add a second placeholder and create a new addition operation:



We can now open a `Session` and `Session.run` the `flexible_addition` tensor to get its value.
However, now we need to pass an additional input, the actual values we want for `x` and `y` at the time of computation:



We can also give multiple inputs to be evaluated:



We can fetch the value of a tensor multiple times, and each time we can give a new value to the placeholder.
If we give it a list of values, then it will use that list as an input.

Notice that the tensor we created for addition uses the `tf.add` method, not the Python `+` method.
In Python, using `+` to "add" two lists actually contatenate them:



The `tf.add` implements vector addition (actually tensor addition).

It is possible to add multiple operations at once:



We can get the value of this tensor for multiple inputs via:



This operation of opening a `Session` performing a computation and then closing it is recurrent in programming in general.
Python has a special syntax to reduce the number of lines it takes to write the above:



The `with` takes care of opening the `Session` and then closing it when the block of indented code finished running.

We can clear the default graph with:




# Variables



Variables are just like placeholders in the sense that they are filled with different values. Unlike placeholders, however, the value of variables are persistent across calls to `session.run`.
TensorFlow uses `tf.Variable` to hold the value of parameters over which we optimize the value of some loss function.

The `tf.Variable` constructor takes one input as argument. This input defines the type of the variable, like an integer, floar or even a list.
Let's create a new variable:



Notice that 4 new operations were added to the graph. These operations provide ways to assign value to the variable and to read the value from it.

If we try to run a session to get the value of `x` we will get an error:



The error says we are "Attempting to use uninitialized value Variable<sub>1</sub>:0".
The value `33` is used to determine the type of the variable, but is not automatically set as the default value of the variable.
Thus, the variable does not have a value when we try `Session.run` on it.
To assign the number `33` as the default value of `x` we need to initialize it:



The first call to the initializer sets the default value of the Variable.
All variables have an `initialize` method to initialize its default value.
Now, because `x` is a Variable, when we call `session.run` again its value is still `33`.
This contrasts to placeholders.
A placeholder needs its value to be set every time we call `session.run`.
We will use placeholders as the input of the neural network.

There is a helper function that initializes all of the variables to their default values at once:



The method `tf.global_variables_initializer` initializes all of the variables in the graph.




# Computing Gradients



One of the main reasons for using TensorFlow is the ability to automatically compute the gradient of a loss function defined on top of a graph.

To do so, we use the method `tf.gradients`.
This method takes two arguments:

1.  `ys`: a tensor or a list of tensors we would like to derivate
2.  `xs`: a tensor or a list of tensors to derivate with respect to

The `tf.gradients` method follows the graph starting at `ys` to compute the derivatives.



Let's execute the graph to get the actual value:



Returns the derivative of $x^2$ with respect to $x$ evaluated at $x=10$.




# Optimization



We will now implement a linear regression using stochastic gradient descent with TensorFlow.

Let's generate some fake data:



Let's divide the data into training, validation and test sets:



We will use the training data to estimate the parameters, and the validate data to estimate the out of sample loss as we improve the parameter estimates.
Now, let's build the linear model graph:



The definition of the placeholder `x_input` takes two arguments: the data type (float) and the shape.
The shape is `None` lines by 10 columns, and the `None` tells TensorFlow that the number of lines can be anything.

If we pass in a vector for `x_input`, then we can obtain the value of the tensor `y_pred`:



If multiples lines are passed to `x_input`, then we get an array of predictions for $y$:



Now, let's add the loss function to the graph:



We can now compute the mean squared error on the training set:



To obtain the value of the parameters (betas) we will use stochastic gradient descent.
TensorFlow has us covered with the operation `tf.train.GradientDescentOptimizer`.
We can use it to directly compute one step of the gradient descent.
The gradient descent method takes as input the learning rate, and provides a method called `minimize`.
This `minimize` method takes as input a tensor representing the loss function, and it then takes one step on the gradient descent algorithm using the gradient values of the loss function.



Each time we fetch the operation above, it takes a step following the gradient descent evaluated on whatever data was passed as input to the model.
Because we do not have a lot of data, we can run the gradient descent using the entire training set.



We can evaluate how well this model does on the test set.



If we were to implement the stochastic gradient descent, then we would feed a single random data point and then update the weights with the gradient descent on the loss function:




# Neural Network



Let's now use the same data to estimate a simple neural network.
First, reset the default graph:



Then, create the 1st layer of the neural network:



Let's add a final layer:



Create the loss function:



Create the gradient descent optimizer:



Minimize the loss function to estimate the parameters:

